In [2]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
import sys,time, random,os
from time import sleep
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
import pandas


# PROXY = "139.255.17.2:47421" # IP:PORT or HOST:PORT

options = Options()
# options.add_argument("--headless")
# options.add_argument('--proxy-server=%s' % PROXY)
# options.headless = True
driver = webdriver.Chrome('chromedriver.exe',options=options)

url = "https://www.yelp.com/search?cflt=nightlife&find_loc=San+Francisco%2C+CA"

driver.get(url)
sleep(5)
html = driver.page_source
lists = driver.find_elements_by_css_selector('.alternate__373c0__1uacp')

# Ads =[]
Ads_links = set()
# normal = []
normal_links = set()

def get_links(lists):
    for i in lists:
        if "Ad" in i.text:
            try:
#                     Ads.append(i.find_element_by_css_selector('.link-size--inherit__373c0__2JXk5').text)
                Ads_links.add(i.find_element_by_css_selector('.link-size--inherit__373c0__2JXk5').get_attribute('href'))
            except:
                continue
        else:
            try:
#                     normal.append(i.find_element_by_css_selector('.link-size--inherit__373c0__2JXk5').text)
                normal_links.add(i.find_element_by_css_selector('.link-size--inherit__373c0__2JXk5').get_attribute('href'))
            except:
                continue

def get_next_page_links():
    try:
        next_link = driver.find_element_by_link_text('Next')
        next_url = next_link.get_attribute('href')  # if removed
        driver.get(next_url)
        next_page = driver.page_source
        lists = driver.find_elements_by_css_selector('.alternate__373c0__1uacp')
    except:
        print("no link found")
        lists[:] = []
    return lists

def link_extraction(links):
    site_urls = []
    for link in links:
        text = ""
        driver.get(link)
        page_html = driver.page_source
        try:
            map_box = driver.find_element_by_css_selector('.mapbox-text')
            elements = map_box.find_elements_by_css_selector('a')
            verify = False
            for i in elements:
                if '.com' in i.text:
                    text = i.text
                    verify = True
                    site_link = driver.find_element_by_link_text(text).get_attribute('href')
                    site_urls.append(site_link)
            if not verify:
                site_urls.append("No Site URL Present")
        except:
            print("found an exception")
            site_urls.append("Found an exception at : "+ link)
            continue
    return site_urls

def extract_url(urls):
    url_list = []
    for url in urls:
        if "No Site URL Present" in url or "Found an exception at : " in url:
            url_list.append(url)
        else:
            site = url.split('&')[0].split('=')[1].replace('%3A',':').replace('%2F','/')
            url_list.append(site)
    return url_list

def convert_to_file(list_of_links,name,column_name):
    df = pandas.DataFrame(list_of_links,columns = [column_name],index = None)
    df.to_csv(name,index = None)

while True:
    get_links(lists)
    lists = get_next_page_links()
    if len(lists) == 0:
        break

Ads_list = list(Ads_links)
convert_to_file(Ads_list,'AP_file.csv','Link')

normal_list = list(normal_links)
convert_to_file(normal_list,'NP_file.csv','Link')

print("Page links fetched")

Ads_URLS = link_extraction(Ads_links)
APU = extract_url(Ads_URLS)
convert_to_file(APU,'APU.csv','Site URL')

print("APU Done")

Normal_URLS = link_extraction(normal_links)
NPU = extract_url(Normal_URLS)
convert_to_file(NPU,'NPU.csv','Site URL')

print("NPU Done")

driver.close()

Page links fetched
https://www.yelp.com/biz_redir?url=http%3A%2F%2Fwww.monroesf.com&website_link_type=website&src_bizid=gipd1MKJBKnYaXtrQbbIJA&cachebuster=1544125056&s=6f051bd69ce44b96c5c34ecc415fb00c633ff91eb289aeddb5d556d02cfda74d&campaign_id=bPr2MdxZfDhybsa5_4h_Jw
http://www.monroesf.com
https://www.yelp.com/biz_redir?url=http%3A%2F%2Fwww.folsomstreetfoundry.com&website_link_type=website&src_bizid=3m-PTR1QzNi07BP1PF4cDA&cachebuster=1544125066&s=33f72ecaa46e10fafa7c83d5742da9ea7dffe2f4665d3e7ff7241db36979c9e5&campaign_id=OBSQzW78tuhqjPBCyz4Jqw
http://www.folsomstreetfoundry.com
https://www.yelp.com/biz_redir?url=http%3A%2F%2FFlore415.com&website_link_type=website&src_bizid=Fc3OLXzSqYp8cM5ixc7jug&cachebuster=1544125072&s=ab64ff8f1ef7c0e05cf8ce7ed643f1faf68f540522bca51f2089f42fcd2d95ca&campaign_id=RVyP9rHX-tjgbxuNmsmDnQ
http://Flore415.com
https://www.yelp.com/biz_redir?url=http%3A%2F%2Fwww.casanovasf.com&website_link_type=website&src_bizid=rfe0wEdflonLJ76oWuNnBw&cachebuster=1544125079

APU Done


KeyboardInterrupt: 